<a href="https://colab.research.google.com/github/grsart/BiomolComp/blob/main/Pratica7/PraticaTranscriptomica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*Prática adaptada deste [colab](https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/325_Transcriptomics_Unveiled.ipynb#scrollTo=bo6AWQz3qnp9)*

Nesta prática, iremos entender como tratar, analisar e interpretar um experimento de transcriptoma.

Especificamente, iremos fazer a análise de dados reais de sequenciamento de RNA de célula única.


## Instalação dos algoritmos

O primeiro passo da prática é instalar os algoritmos necessários para a prática.

### ScanPy
Biblioteca Python especializada em análises de scRNA-seq

### AnnData
Biblioteca de estruturação de dados, desenvolvida para organizar e integrar diversos tipos de dados genômicos.
Dividido em funções de Matrizes de Dados e Anotação

In [ ]:
!pip install scanpy
!pip install anndata

## Dados a serem utilizados

Inicialmente, vamos fazer o download dos dados de scRNA-seq que vamos utilizar.

### Descrição do experimento
Esses dados foram gerados pelo sequenciamento de mRNAs presentes em diferentes regiões de um tecido. Além disso, a anotação espacial da origem do mRNA permite a determinação do perfil de expressão gênica para cada localização espacial de um determinado tecido.

O sequenciamento acontece por meio de algumas etapas:
- **Preparação de biblioteca:** Amplificação de RNA e anotação espacial
- **Sequenciamento:** Sequenciamento da biblioteca gerada e tratamento dos dados gerados

### Organização da base de dados
O arquivo está depositado neste [endereço](https://cell2location.cog.sanger.ac.uk/tutorial/mouse_brain_visium_wo_cloupe_data.zip).

O arquivo compactado está dividido basicamente em duas pastas. A primeira, *raw*, contém os dados originais de sequenciamento. Já a segunda possui a versão tratada dos dados, após passar por etapas de processamento de anotação espacial, quantificação de expressão gênica, normalização e filtragem por meio de controle de qualidade.


In [ ]:
# Baixando o arquivo com os dados
!rm mouse_brain_visium_wo_cloupe_data
!wget https://cell2location.cog.sanger.ac.uk/tutorial/mouse_brain_visium_wo_cloupe_data.zip

# Descompactando o arquivo
!unzip mouse_brain_visium_wo_cloupe_data.zip

# Definindo a pasta contendo os dados que serão utilizados
sp_data_folder = './mouse_brain_visium_wo_cloupe_data/rawdata/ST8059048/'




In [22]:
#Importando as bibliotecas necessárias para as análises
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc

import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:
# Carregando o arquivo bruto proveniente do sequenciamento

adata = sc.read_visium(sp_data_folder, count_file='filtered_feature_bc_matrix.h5', load_images=True)

**Questão 1:** A saída do comando anterior gerou um aviso de alerta. Qual foi o alerta e por que acha que isso aconteceu?

Corrija o problema anterior com o seguinte comando:

In [ ]:
adata.var_names_make_unique()

### Visualizando e compreendendo os dados existentes neste experimento

Agora, vamos ver como o arquivo foi carregado.

**Questão 2:** A partir do resultado abaixo, quais informações podem ser retiradas? O que são os observáveis? E as variáveis? Quantos observáveis e variáveis existem neste arquivo? Justifique


In [ ]:
adata

Vamos olhar com mais carinho para os observáveis e para as variáveis. Primeiramente, vamos ver as primeiras cinco entradas dos observáveis.



In [ ]:
adata.obs.head()


In [ ]:
adata.var.head()


Quais células expressam qual gene?

A matriz X trás essa informação, contendo o dado de expressão de cada gene para cada célula da amostra.

In [ ]:
# Convert the sparse matrix to a dense matrix
dense_matrix = adata.X.toarray()

# Print the dense matrix
print(dense_matrix)

**Questão 3:** Quais informações pode tirar desta tabela? É algo esperado para um estudo de expressão a nível celular?

Vamos extrair outra informação agora.
- Quantos e quais são os genes que são efetivamente expressos em uma determinada célula?
- O número de genes expressos mudam de célula para célula?

Avalie o número de genes expressos para diferentes células, alterando o valor de 'cell_index' e rodando novamente a célula de execução abaixo.

In [ ]:
cell_index = 0  # Índice da célula de interesse

# Obter os dados de expressão da célula de interesse
gene_expression = adata.X[cell_index]

# Obter os índices somente dos genes que possuem um valor de expressão diferente de zero naquela célula
expressed_gene_indices = gene_expression.nonzero()[1]

# Obter os nomes dos genes para os índices recuperados anteriormente
expressed_genes = adata.var_names[expressed_gene_indices]

# Imprimir a lista dos genes expressos para aquela célula
print(expressed_genes)

Será que existe algum gene que é consistentemente mais expresso neste tecido? Podemos avaliar isso com um único comando:

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

Como foi dito no começo da prática, esses dados são provenientes de análise single-cell em um tecido. Uma informação existente nos dados é exatamente a imagem do tecido como um todo:

In [ ]:
sc.pl.spatial(adata, img_key="hires", color=None)

Além da imagem do 'branco', é possível ver os pontos onde foram coletadas as amostras

In [ ]:
sc.pl.spatial(adata, img_key="hires", alpha=0.5) #Change alpha to 0 to see the tissue sample or plot by setting color=None

Os dados também fornecem a informação de quanto de cada gene é expresso em cada célula (cada esfera destacada no gráfico anterior). Podemos então visualizar essa distribuição.
  
Vamos fazer isso para três genes de interesse. O primeiro é o gene mais expresso, como vimos no gráfico de barras anteriormente.
  
Outros dois são os genes **Rorb** e **Vip**.
  
Antes, vamos checar se esses genes estão presentes nos dados

In [ ]:
# Listando o nome de todos os genes existentes nas leituras
gene_names = adata.var.index

# Verificando se os genes Rorb e Vip estão presentes na lista gerada
print("Is Rorb gene present in the vars?", "Rorb" in gene_names)
print("Is Vip gene present in the vars?", "Vip" in gene_names)

**Questão 4:** O que a análise acima significa?

Vamos agora projetar os resultados de perfil de expressão dos três genes no tecido

In [ ]:
with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'black'}):
  sc.pl.spatial(adata, color=["Rorb", "Vip", "mt-Co3"], img_key=None, size=1,
                    vmin=0, cmap='magma', vmax='p99.0',
                    gene_symbols='SYMBOL'
                  )

**Questão 5:** Discuta os gráficos obtidos. Existe alguma região onde os três genes estão expressos? Existe alguma região que algum gene não é expresso? Consegue ver algum padrão comum entre os gráficos?

### Preparação e limpeza dos dados

Uma vez que a estrutura de dados foi compreendida e visto que os dados podem ser avaliados e analisados, vamos começar a curar e refinar os dados de forma a responder a nossa questão biológica - Como diferentes genes se agrupam e têm perfil de expressão correlacionados em localidades distintas de um mesmo tecido?

Neste experimento, existe um interesse maior em genes ribossomais e mitocondriais. Para facilitar o processo de filtragem e avaliação em etapas posteriores, pode-se criar novas colunas sinalizando se o referido gene é mitocondrial ou ribossomal ou nenhum.
  
O primeiro passo é identificar esses genes. Para o mitocondrial, basta buscar no seu nome aqueles que se iniciam com ***'mt_'***


In [ ]:
# Criando uma nova coluna para sinalizar se um gene é mitocondrial ou não
adata.var["mt"] = adata.var_names.str.startswith("mt-")

#Identificando os genes que são mitocondriais
adata.var[adata.var.index.str.startswith('mt-')]

**Questão 6:** Quandos genes mitocondriais foram identificados?

Os genes ribossomais não têm uma marcação em seu nome. Felizmente, esses genes são conhecidos e tabelados em bancos de dados de expressão gênica. Vamos obter essa lista e usá-la para buscar nos nossos dados quais são os genes ribossomais.

In [ ]:
# Download da lista de genes ribossomais
!wget -O KEGG_RIBOSOME.v2023.1.Hs.csv "https://www.gsea-msigdb.org/gsea/msigdb/human/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=CSV"

# Carregando o arquivo baixado como uma lista no python
ribo_genes = pd.read_csv('./KEGG_RIBOSOME.v2023.1.Hs.csv', skiprows=2, header=None)

# Imprimindo os genes ribossomais obtidos do banco de dados GSEA
print(ribo_genes)

# Identificando os genes ribossomais, comparando a lista de referencia com os dados experimentais
adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)

Vamos agora avaliar a qualidade dos dados experimentais, analisando as características dos genes mitocondriais e ribossomais. Isso permite avaliar aspectos da qualidade celular, como conteudo mitocondrial e atividade ribossomal. Isso é um bom indicativo da saúde da célula, integridade do RNA e evita artefatos experimentais.

Para isso, vamos calcular o QC (Quality Control) levando em conta os genes mitocondriais e ribossomais

In [ ]:
# Cálculo dos parâmetros QC de célula
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", 'ribo'], inplace=True)

# Verificação se os parâmetros foram calculados
adata

**Questão 7:** A partir do resultado acima, como você sabe se os parâmetros vinculados ao QC foram calculados?

Vamos verificar alguns parâmetros de interesse.
  
- Quantas células/spots foram analisadas?  
- Quantas leituras foram feitas por célula, em média?  
- Quantos genes foram encontrados por célula, em média?  

In [ ]:
# Obtendo número de amostras
number_of_spots = adata.obs_names.shape[0]

# Calculando o número de leituras médio por amostra
mean_reads_per_spot = adata.obs['total_counts'].mean()

# Calculando o número mediano de genes por amostra
median_genes_per_spot = adata.obs['n_genes_by_counts'].median()

print("Number of spots under tissue:", number_of_spots)
print("Mean reads per spot:", mean_reads_per_spot)
print("Median genes per spot:", median_genes_per_spot)

Vamos avaliar do ponto de vista dos genes.

Neste caso, informações de interesse dizem respeito a quantas amostras possuem determinado gene, ou a porcentagem de genes efetivamente expressos em uma célula.  

In [ ]:
adata.var

**Questão 8:** O que você acha que significam as as colunas *n_cells_by_counts* e *total_counts* ?

Vamos olhar mais profundamente para as amostras

In [ ]:
adata.obs

- Como é a distribuição de número de leituras por células? Quantas células tem quantas leituras?
- Quantas células têm quantos genes expressos?

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 4))
sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0])
sns.histplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[1])

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
             jitter=0.4, multi_panel=True)

Células que tenham uma quantidade muito grande ou muito pequena de leituras ou que tenham uma quantidade bem menor de genes expressos devem ser descartadas para não introduzir artefatos nas análises de perfil de expressão. Como dito anteriormente, podemos também filtrar baseado na porcentagem de genes ribossomais/mitocondriais expressos.

**Questão 9:** Baseado nessas informações e nos gráficos acima, quais seriam os valores de corte que você sugeriria para para descartar células?


Para este caso, foram definidos os filtros de:
- intervalo de *counts*: 5000 < counts < 35000
- % gene mitocondrial < 20
- % gene ribossomal < 2

**Questão 10:** Estes cortes fazem sentido baseado no gráfico de violino?




In [ ]:
sc.pp.filter_cells(adata, min_counts=5000)
sc.pp.filter_cells(adata, max_counts=35000)
adata = adata[adata.obs["pct_counts_mt"] < 20]
adata = adata[adata.obs["pct_counts_ribo"] < 2]


Vamos visualizar como ficou o panorama de células após o filtro


In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
             jitter=0.4, multi_panel=True)

**Questão 11:** Também é recomendável remover aqueles genes que aparecem em poucas ou nenhuma célula. Qual seria o racional por trás disso?

In [ ]:
# remover os genes que foram detectados em menos de 10 células
sc.pp.filter_genes(adata, min_cells=10)

### Tratamento de amostrar
Após o processo de filtragem, alguns pré-processamentos precisam ser realizados de forma a eliminar vieses de análise.

O primeiro ponto relevante é a normalização do´número de leituras entre diferentes células. Isso precisa ser feito pois cada célula tem um número diferente de leituras, mesmo que tenha a mesma quantidade relativa de genes expressos.

O segundo ponto é tomar o logaritmo do valor de count. Esse processo é importante para reduzir a variância e normalizar a distribuição dos dados.

Vamos comparar o efeito desses dois procedimentos no gene de maior count, identificado anteriormente como sendo o mt-Co3.

In [ ]:
# Distribuição de contagem antes de qualquer manipulação dos dados

# Definir o nome do gene
gene_name = 'mt-Co3'

# Obter o índice do gene na lista
gene_index = list(adata.var_names).index(gene_name)

# Extrair a contagem de expressão gênica para este gene
gene_counts = adata.X[:, gene_index]

# Converter a matriz esparsa em um array
gene_counts_array = gene_counts.toarray().flatten()

# Gerar o histograma
plt.hist(gene_counts_array, bins=30)
plt.title(f"Histogram of Counts per Cell for Gene {gene_name}")
plt.xlabel("Counts per Cell")
plt.ylabel("Frequency")
plt.show()

Agora vamos fazer o mesmo, mas para o dado normalizado, para o mesmo gene

In [ ]:
# Normalizando os dados
sc.pp.normalize_total(adata, inplace=True, target_sum=1e4)

# Obter a contagem de expressão gênica normalizada para o gene de interesse
gene_counts_normalized = adata.X[:, gene_index]

# Converter a matriz esparsa em um array
gene_counts_normalized_array = gene_counts_normalized.toarray().flatten()

# Gerar o histograma
plt.hist(gene_counts_normalized_array, bins=30)
plt.title(f"Histogram of Normalized Counts per Cell for Gene {gene_name}")
plt.xlabel("Normalized Counts per Cell")
plt.ylabel("Frequency")
plt.show()

Vamos agora tirar o logaritmo dos dados  repetir o processo

In [ ]:
# Transformando os dados em logaritmo
sc.pp.log1p(adata)

# Obter a contagem de expressão gênica em log para o gene de interesse
gene_counts_log_transformed = adata.X[:, gene_index]

# Converter a matriz esparsa em um array
gene_counts_log_transformed_array = gene_counts_log_transformed.toarray().flatten()

# Gerar o histograma
plt.hist(gene_counts_log_transformed_array, bins=30)
plt.title(f"Histogram of Log-Transformed Counts per Cell for Gene {gene_name}")
plt.xlabel("Log-Transformed Counts per Cell")
plt.ylabel("Frequency")
plt.show()

**Questão 12:** Discuta a diferença entre os gráficos e como as alterações podem afetar na análise relativa de perfil de expressão

### Identificando genes com perfil de expressão altamente variável
Esses genes em geral têm perfil de expressão diferente em diferentes tipos de célula ou de condição biológica. Por conta disso, pode ser utilizado como marcado para aquele tipo de célula.



In [ ]:
# Identificação dos top-2000 genes mais variáveis
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)

adata.var

Note o surgimento de uma nova coluna booleana, nomeada "Highly_variable", indicando se aquele gene é um dos top-2000 mais variáveis.  
  
Uma outra forma de visualizar esse tipo de informação é graficando a dispersão do perfil de expressão do gene


In [ ]:
sc.pl.highly_variable_genes(adata)

**Questão 13:** Qual a importância de identificar os genes mais variáveis durante este experimento? Posso descartar os genes que estão fora da lista top-2000? Justifique

In [ ]:
# Extrair somente os dados mais variáveis, descartando os outros
adata = adata[:, adata.var.highly_variable]

Este primeiro filtro já reduziu consideravelmente a dimensionalidade dos dados. Porém, 2000 variáveis ainda é um número bem elevado.
  
O próximo passo é realizar uma outra redução de dimensionalidade, tentando descrever a variância do sistema com o menor número possível de combinações de variáveis. Para isso, utiliza-se a Análise de Componentes Principais.



In [ ]:
sc.pp.pca(adata) #By default calculates 30 PCAs

# Graficando a variância de cada combinação de variável
sc.pl.pca_variance_ratio(adata, log=True)

**Questão 14:** Visualizando o gráfico, qual seria o seu critério para definir quantas componentes principais serão utilizadas para o restante da análise?

Após a definição do número 20 como adequado para a redução de dimensionalidade, é importante calcular a informação das vizinhanças utilizando o perfil de expressão gênica das diferentes células.  
  
Neste cálculo, busca-se observar as correlação no perfil de expressão e encontrar as células com perfis mais parecidos

In [ ]:
sc.pp.neighbors(adata, n_pcs=20)

Um outro nível de redução de dimensionalidade é indicado, de forma a reduzir para duas ou três dimensões. A abordagem UMAP (Uniform Manifold Approximation and Projection) reduz a dimensionalidade preservando as estruturas locais e globais dos dados, importante para manter a visualização dos dados

In [ ]:
# Reduzindo a dimensionalidade de 20 para 2 utilizando o algoritmo UMAP
sc.tl.umap(adata, n_components=2)

# Graficando o resultado:
sc.pl.umap(adata)

A forma de deixar o gráfico mais palatável é projetar as informações dos genes expressos em mais células, dentre aqueles altamente variáveis.  
  
Para isso, deve-se identificar esses genes e então projetar eles no gráfico UMAP obtido acima

In [ ]:
# Identificando os genes que foram expressos em uma quantidade maior de células
sorted_by_num_cells = adata.var['n_cells_by_counts'].sort_values(ascending=False)
sorted_by_num_cells



In [ ]:
# Colorindo os gráficos UMAP de acordo com os genes mais expressos
sc.pl.umap(adata, color=["Gm42418", "Fth1", "3110035E14Rik"])

**Questão 15:** Você é capaz de observar alguma variação na expressão para esses três genes? Eles parecem estar correlacionados entre si?



Essa correlação é ajudada quando entendemos quais as células são mais parecidas entre si. A organização das células pode ser feita utilizando clusterização, onde as células com menor distância entre si definem um cluster.


In [ ]:
#Instalando o algoritmo de clusterização
!pip install leidenalg

#Clusterizando os dados. Utilizando como distribuição de dados o gráfico UMAP
sc.tl.leiden(adata, resolution=0.6, key_added="clusters")

# Graficando as informações de número de leituras, número de genes expressos por amostra e a predição de clusterização
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)

Relembrando novamente, este experimento foi feito avaliando múltiplas células em um tecido. Uma das informações experimentais gravada é a posição relativa das células no tecido. Podemos então projetar as três informações anteriores na distribuição espacial, na tentativa de encontrar algum padrão espacial de similaridade de expressão gênica


In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["total_counts", "n_genes_by_counts","clusters"])


**Questão 16:** Como você explica esses gráficos? Quais as informações mais relevantes?  

No estágio atual, foram identificados 12 clusters, que possuem perfis de expressão gênica similares. Uma boa pergunta a se fazer é: Qual é o gene representativo para cada um dos clusteres? Em outras palavras, qual gene poderia ser o marcador de um tipo celular ou um estado biológico?  
  
A análise neste momento é fazer uma comparação entre os perfis de expressão gênica de clusters diferentes para buscar os genes mais diferentes entre eles.  
  
Essa análise passa por métodos estatísticos como t-test para avaliar a significância estatística da diferença

In [ ]:
# Agrupar as celulas de mesmo cluster e executar análise de expressão gênica diferencial usando t-test
sc.tl.rank_genes_groups(adata, "clusters", method="t-test")

#Plot dos genes diferenciais de cada cluster frente a todos os outros.
sc.pl.rank_genes_groups(adata, n_genes=10)

**Questão 17:** Como você interpreta os gráficos? É possível definir genes marcadores para cada cluster?

Uma vez identificados genes promissores, é possível ver como a sua expressão ocorre nas outras células. Trata-se de uma análise visual das diferenças da expressão daquele gene em células e clusters distintos, assim como possibilita identificar similaridade entre clusters


In [ ]:
#Avaliação dos 10 genes com perfil de expressão mais diferentes do cluster 6 em relação aos demais
sc.pl.rank_genes_groups_heatmap(adata, groups="6", n_genes=10, groupby="clusters")

**Questão 18:** Qual o gene chamou mais atenção? Justifique e discuta as implicações biológicas.


Podemos também avaliar como é a relação de perfil de expressão desse gene e a localização espacial no tecido em análise.



In [ ]:
gene='ESCREVER GENE AQUI DA QUESTÂO 18 AQUI'
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["clusters", gene])